# High-level Keras (TF) Example

In [1]:
import os
import sys
import numpy as np
os.environ['KERAS_BACKEND'] = "tensorflow"
import keras as K
import tensorflow
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, Dropout
from common.params import *
from common.utils import *

Using TensorFlow backend.


In [2]:
# Force one-gpu
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [3]:
# Performance Improvement
# 1. Make sure channels-first (not last)
K.backend.set_image_data_format('channels_first')

In [4]:
print("OS: ", sys.platform)
print("Python: ", sys.version)
print("Keras: ", K.__version__)
print("Numpy: ", np.__version__)
print("Tensorflow: ", tensorflow.__version__)
print(K.backend.backend())
print(K.backend.image_data_format())
print("GPU: ", get_gpu_name())
print(get_cuda_version())
print("CuDNN Version ", get_cudnn_version())

OS:  linux
Python:  3.5.2 |Anaconda custom (64-bit)| (default, Jul  2 2016, 17:53:06) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
Keras:  2.1.4
Numpy:  1.14.1
Tensorflow:  1.4.0
tensorflow
channels_first
GPU:  ['Tesla P100-PCIE-16GB', 'Tesla P100-PCIE-16GB']
CUDA Version 8.0.61
CuDNN Version  6.0.21


In [5]:
def create_symbol(n_classes=N_CLASSES):
    model = Sequential()
    
    model.add(Conv2D(50, kernel_size=(3, 3), padding='same', activation='relu',
                     input_shape=(3, 32, 32)))
    model.add(Conv2D(50, kernel_size=(3, 3), padding='same', activation='relu'))    
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(100, kernel_size=(3, 3), padding='same', activation='relu'))
    model.add(Conv2D(100, kernel_size=(3, 3), padding='same', activation='relu'))    
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.25))
        
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(n_classes, activation='softmax'))
    return model

In [6]:
def init_model(m, lr=LR, momentum=MOMENTUM):
    m.compile(
        loss = "categorical_crossentropy",
        optimizer = K.optimizers.SGD(lr, momentum),
        metrics = ['accuracy'])
    return m

In [7]:
%%time
# Data into format for library
x_train, x_test, y_train, y_test = cifar_for_library(channel_first=True, one_hot=True)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)
print(x_train.dtype, x_test.dtype, y_train.dtype, y_test.dtype)

Preparing train set...
Preparing test set...
(50000, 3, 32, 32) (10000, 3, 32, 32) (50000, 10) (10000, 10)
float32 float32 int32 int32
CPU times: user 666 ms, sys: 592 ms, total: 1.26 s
Wall time: 1.26 s


In [8]:
%%time
# Load symbol
sym = create_symbol()

CPU times: user 329 ms, sys: 360 ms, total: 688 ms
Wall time: 689 ms


In [9]:
%%time
# Initialise model
model = init_model(sym)

CPU times: user 129 ms, sys: 16.7 ms, total: 146 ms
Wall time: 144 ms


In [10]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 50, 32, 32)        1400      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 50, 32, 32)        22550     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 50, 16, 16)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 50, 16, 16)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 100, 16, 16)       45100     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 100, 16, 16)       90100     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 100, 8, 8)         0         
__________

In [11]:
%%time
# Main training loop: 1m16s
model.fit(x_train,
          y_train,
          batch_size=BATCHSIZE,
          epochs=EPOCHS,
          verbose=1)

Epoch 1/10
50000/50000 [==============================] - 8s 169us/step - loss: 1.8493 - acc: 0.3242
Epoch 2/10
50000/50000 [==============================] - 8s 154us/step - loss: 1.3995 - acc: 0.4903
Epoch 3/10
50000/50000 [==============================] - 8s 150us/step - loss: 1.1655 - acc: 0.5832
Epoch 4/10
50000/50000 [==============================] - 8s 152us/step - loss: 1.0133 - acc: 0.6411
Epoch 5/10
50000/50000 [==============================] - 7s 147us/step - loss: 0.8987 - acc: 0.6823
Epoch 6/10
50000/50000 [==============================] - 7s 146us/step - loss: 0.8078 - acc: 0.7140
Epoch 7/10
50000/50000 [==============================] - 7s 148us/step - loss: 0.7291 - acc: 0.7455
Epoch 8/10
50000/50000 [==============================] - 7s 148us/step - loss: 0.6743 - acc: 0.7637
Epoch 9/10
50000/50000 [==============================] - 8s 153us/step - loss: 0.6171 - acc: 0.7813
Epoch 10/10
50000/50000 [==============================] - 8s 150us/step - loss: 0.5671 - a

In [12]:
%%time
# Main evaluation loop
y_guess = model.predict(x_test, batch_size=BATCHSIZE)
y_guess = np.argmax(y_guess, axis=-1)
y_truth = np.argmax(y_test, axis=-1)

CPU times: user 348 ms, sys: 85 ms, total: 433 ms
Wall time: 466 ms


In [13]:
print("Accuracy: ", 1.*sum(y_guess == y_truth)/len(y_guess))

Accuracy:  0.7679
